In [ ]:
# Install required python packages
!pip3 install --upgrade --user google-cloud-aiplatform

In [1]:
# restart the kernet after libraries are loaded
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# Task 2
# import software libraries required
import requests
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

In [6]:
# Task 2.1
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
model = GenerativeModel("gemini-1.0-pro")

prompt = "Why is the sky blue?."


responses=model.generate_content(prompt, stream=True)

print("-------Prompt--------")
print(prompt)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")



-------Prompt--------
Why is the sky blue?.

-------Response--------
The sky appears blue due to a phenomenon called Rayleigh scattering. Sunlight, which is white light, is composed of all colors of the rainbow. When sunlight enters the Earth's atmosphere, it interacts with the tiny molecules of air, such as nitrogen and oxygen. These molecules scatter the light in all directions. 

However, blue light has a shorter wavelength than other colors, so it is scattered more strongly by the air molecules. This means that more blue light reaches our eyes from the sky than any other color. As a result, the sky appears blue to us.

Here are some additional details about the scattering of light:

* The amount of scattering depends on the size of the particles and the wavelength of light. Smaller particles scatter shorter wavelengths more strongly, while larger particles scatter longer wavelengths more strongly.
* The intensity of scattered light is inversely proportional to the fourth power of t

In [7]:
# Task 2.2
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "Location"
            }
        }
    },
)

In [8]:
# Task 2.3
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

In [9]:
# Task 2.4
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
prompt = "What is the weather like in Boston?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Boston"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0950130522
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0840406194
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.151906386
    severity: HARM_SEVERITY_LOW
    severity_score: 0.207535744
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.138578326
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0826973394
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    pro

In [10]:
# Task 3
# Run the following cell to import required libraries 
from vertexai.preview.generative_models import (
    GenerationConfig,
    Image,
    Part,
)

In [11]:
# Task 3.1
# Load the correct Gemini model use the following documentation to assist:
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview#supported-use-cases
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

In [12]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_images([content])
    return True


def display_content_as_video(content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=600))
    return True


def print_multimodal_prompt(contents: list[str | Image | Part]):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)

In [ ]:
# Task 3.2 Generate a video description
# In this cell, update the prompt to ask Gemini to describe the video URL referenced.
# You can use the documentation at the following link to assist.
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/sdk-for-gemini/gemini-sdk-overview-reference#generate-content-from-video
# 
# Video URI: gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4
# 
prompt = """
What is shown in this video?
Where should I go to see it?
What are the top 5 places in the world that look like this?
"""
video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")

-------Prompt--------

What is shown in this video?
Where should I go to see it?
What are the top 5 places in the world that look like this?




-------Response--------
